In [9]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
from datetime import date, timedelta
from scipy.stats.stats import spearmanr 
from scipy import stats
from pandas import *
import re
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
import re
!pip install us
import us
plt.style.use(['seaborn-darkgrid'])

import warnings
warnings.filterwarnings('ignore')

# Preprocessing

In [316]:
# Set datetime index
# Filter out primary elections 
def make_pretty_RCP(CSV_list):
    clean_dfs = []
    for csv in CSV_list:
        df = read_csv(csv)
        df = df.drop(['Unnamed: 0'], axis=1)
        df['Date'] = to_datetime(df['Date'])
        df['Year'] = df['Year'].astype(int)
        df.columns = ['Race','Poll','Result','Spread','Date','Year']
        df = df[df['Race'].str.contains('Primary')==False]
        clean_dfs.append(df)
    return clean_dfs

clean_dfs = make_pretty_RCP(['RCP_governor.csv','RCP_senate.csv','RCP_house.csv'])
gov_df, sen_df, house_df = clean_dfs[0], clean_dfs[1], clean_dfs[2]

# This format is consistent for all dfs
gov_df.head()

,Race,Poll,Result,Spread,Date,Year
0,Colorado Governor - McInnis vs. Hickenlooper,Rasmussen Reports,"Hickenlooper 42, McInnis 48",McInnis +6,2010-04-16,2010
3,New York Governor - Levy vs. Cuomo,Siena,"Cuomo 58, Levy 23",Cuomo +35,2010-04-18,2010
4,New York Governor - Lazio vs. Cuomo,Siena,"Cuomo 61, Lazio 24",Cuomo +37,2010-04-18,2010
5,Texas Governor - Perry vs. White,Rasmussen Reports,"Perry 48, White 44",Perry +4,2010-04-19,2010
6,Massachusetts Governor - Baker vs. Patrick vs....,Western NE College,"Patrick 34, Baker 27, Cahill 29",Patrick +5,2010-04-19,2010


In [317]:
def make_pretty_Winners(CSV_list):
    win_dfs = []
    for csv in CSV_list:
        df = read_csv(csv)
        df = df.drop(['Unnamed: 0'], axis=1)
        df['date'] = to_datetime(df['date'])
        df = df.set_index('date')
        df['lastname'] = [i[-1:][0] for i in df['name'].str.split()]
        # JI is our join index
        df['JI'] = df['lastname']+' '+df['state']

        # This is the part we care about in the aggregation
        win = df[['party','JI']]
        win_dfs.append(win)
    return win_dfs, pd.concat(win_dfs)

# all_win is winners from all races together in a single df
win_dfs, all_win = make_pretty_Winners(['Clean_Governor_Winners.csv','Clean_Senate_Winners.csv','Clean_House_Winners.csv'])
gov_win, sen_win, house_win = win_dfs[0], win_dfs[1], win_dfs[2]

# This format is consistent for all dfs
gov_win.head()

,party,JI
date,,
2000-12-15,r,hoeven ND
2000-12-21,r,perry TX
2000-12-21,r,perry TX
2002-12-02,r,lingle HI
2003-01-01,d,richardson NM


In [305]:
# Helper function for classifying party
# 0 for Republican, 1 for Democrat, nans stay as is
def party_class(df):
    ind = []
    for i in df['party']:
        if i=='r':
            ind.append(0)
        elif i=='d':
            ind.append(1)
        else:
            ind.append(np.nan)
    return ind

In [306]:
# Dictionary mapping states names to abbreviations
states_dict = us.states.mapping('name', 'abbr')

# Party Classification for Gubernatorial, Senatorial, and Congressional Races

In [353]:
def Party_Classification(RCP_df, win_df, states_dict):
    '''
    RCP_df is RealClearPolitics polling data  
    win_df is data about past winners
    Both are formatted as shown in preprocessing
    
    states_dict is a dictionary mapping state names to abbreviation; shown in preprocessing
    '''
    # Create column for poll leaders
    RCP_df['Lead'] = [i[0] for i in RCP_df['Spread'].str.split()]
    RCP_df['Lead'] = RCP_df['Lead'].str.lower()
    
    # Create column for state name
    # The case we have congressional polls
    if RCP_df['Race'][0].find('District') != -1:
        RCP_df['State'] = RCP_df['Race'].str.findall(r"^[A-Za-z ]+ ").str[0].str.strip() 
    # The case we have gubernatorial/senatorial polls
    else:
        race_type = RCP_df['Race'][0].split()[1]
        RCP_df['State'] = [i[:i.index(race_type)-1] for i in RCP_df['Race']]   
    
    # Map state names to abbreviations and create join index (JI)
    RCP_df["State"].replace(states_dict, inplace=True)
    # Weird edge case with New Hampshire (in certain places it's N.H.)
    # Not sure what to do outside of hard coding for it
    RCP_df['State'] = RCP_df['State'].fillna('NH')
    RCP_df['JI'] = RCP_df['Lead']+' '+RCP_df['State']
    
    # Join RCP_df to win_df on JI
    RCP_df = RCP_df.set_index('JI').join(win_df.set_index('JI')).set_index('Date')
    RCP_df = RCP_df.reset_index().set_index(['Date', 'State'])
    
    # Democrat=1, Republican=0, nans stay as is
    ind = party_class(RCP_df)
    RCP_df['Party Class'] = ind
    RCP_df.rename(columns={'party':'Party'}, inplace=True)
    
    return RCP_df

In [345]:
# Fewer nans if we use all_win versus sen_win
# Makes sense -- winners of one office often run for something else down the line (ie. Congressman wants to become a Senator)

# Tradeoffs:
# We have high confidence that the sen_win example is correct, but we're limited to winners who've held that specific office in the past
# In the all_win example we can map to winners who've held ANY office in the past -- this makes our data richer, but we're slightly
# more prone to error (ie. candidate switched parties between races)

# I think using all_win is better, but we should discuss this further
print(Party_Classification(sen_df, sen_win, states_dict)['Party'].isnull().sum())
print(Party_Classification(sen_df, all_win, states_dict)['Party'].isnull().sum())

967
742


### Classified Gubernatorial polling data

In [346]:
gov_df_c = Party_Classification(gov_df, all_win, states_dict)
gov_df_c

Race  \
Date       State                                               
2013-07-02 TX              Texas Governor - Abbott vs. Davis   
2013-10-02 TX              Texas Governor - Abbott vs. Davis   
2013-11-05 TX              Texas Governor - Abbott vs. Davis   
           TX              Texas Governor - Abbott vs. Davis   
2014-02-24 TX              Texas Governor - Abbott vs. Davis   
2014-03-05 TX              Texas Governor - Abbott vs. Davis   
2014-04-15 TX              Texas Governor - Abbott vs. Davis   
2014-06-12 TX              Texas Governor - Abbott vs. Davis   
2014-07-28 TX              Texas Governor - Abbott vs. Davis   
2014-08-08 TX              Texas Governor - Abbott vs. Davis   
2014-09-11 TX              Texas Governor - Abbott vs. Davis   
2014-10-01 TX              Texas Governor - Abbott vs. Davis   
2014-10-03 TX              Texas Governor - Abbott vs. Davis   
2014-10-10 TX              Texas Governor - Abbott vs. Davis   
2014-10-20 TX              Texas Governor - Abbott vs. Davis   
2014-10-23 TX              Texas Governor - Abbott vs. Davis   
2014-10-27 TX              Texas Governor - Abbott vs. Davis   
2018-04-18 TX             Texas Governor - Abbott vs. Valdez   
           TX              Texas Governor - Abbott vs. White   
2018-05-22 TX              Texas Governor - Abbott vs. White   
           TX             Texas Governor - Abbott vs. Valdez   
2018-05-30 TX             Texas Governor - Abbott vs. Valdez   
2018-06-25 TX             Texas Governor - Abbott vs. Valdez   
2018-07-09 TX             Texas Governor - Abbott vs. Valdez   
2018-07-29 TX             Texas Governor - Abbott vs. Valdez   
2018-08-01 TX             Texas Governor - Abbott vs. Valdez   
2018-08-22 TX             Texas Governor - Abbott vs. Valdez   
2018-09-16 TX             Texas Governor - Abbott vs. Valdez   
2018-09-18 TX             Texas Governor - Abbott vs. Valdez   
2018-10-08 TX             Texas Governor - Abbott vs. Valdez   
...                                                      ...   
2014-08-06 PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-08-28 PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-09-05 PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-09-11 PA       Pennsylvania Governor - Corbett vs. Wolf   
           PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-09-21 PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-09-25 PA       Pennsylvania Governor - Corbett vs. Wolf   
           PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-09-26 PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-10-07 PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-10-10 PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-10-18 PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-10-27 PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-10-29 PA       Pennsylvania Governor - Corbett vs. Wolf   
           PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-10-30 PA       Pennsylvania Governor - Corbett vs. Wolf   
2014-11-01 PA       Pennsylvania Governor - Corbett vs. Wolf   
2018-03-29 PA     Pennsylvania Governor - Ellsworth vs. Wolf   
           PA         Pennsylvania Governor - Mango vs. Wolf   
           PA        Pennsylvania Governor - Wagner vs. Wolf   
2018-04-13 PA         Pennsylvania Governor - Mango vs. Wolf   
           PA     Pennsylvania Governor - Ellsworth vs. Wolf   
           PA        Pennsylvania Governor - Wagner vs. Wolf   
2018-06-14 PA        Pennsylvania Governor - Wagner vs. Wolf   
2018-06-27 PA        Pennsylvania Governor - Wagner vs. Wolf   
2018-08-23 PA        Pennsylvania Governor - Wagner vs. Wolf   
2018-08-30 PA        Pennsylvania Governor - Wagner vs. Wolf   
2018-09-20 PA        Pennsylvania Governor - Wagner vs. Wolf   
2018-09-21 PA        Pennsylvania Governor - Wagner vs. Wolf   
2018-09-27 PA        Pennsylvania Governor - Wagner vs. Wolf   

                       

### Classified Senatorial polling data

In [347]:
sen_df_c = Party_Classification(sen_df, all_win, states_dict)
sen_df_c

,,Race,Poll,Result,Spread,Year,Lead,Party,Party Class
Date,State,,,,,,,,
2012-04-19,MO,Missouri Senate - Akin vs. McCaskill,Rasmussen Reports,"Akin 48, McCaskill 43",Akin +5,2012,akin,r,0.0
2012-07-28,MO,Missouri Senate - Akin vs. McCaskill,Post-Dispatch/Mason-Dixon,"McCaskill 44, Akin 49",Akin +5,2012,akin,r,0.0
2012-08-01,MO,Missouri Senate - Akin vs. McCaskill,Rasmussen Reports,"McCaskill 44, Akin 47",Akin +3,2012,akin,r,0.0
2012-08-13,MO,Missouri Senate - Akin vs. McCaskill,SurveyUSA*,"McCaskill 40, Akin 51",Akin +11,2012,akin,r,0.0
2012-08-21,MO,Missouri Senate - Akin vs. McCaskill,PPP (D),"McCaskill 43, Akin 44",Akin +1,2012,akin,r,0.0
2012-09-18,MO,Missouri Senate - Akin vs. McCaskill,Gravis Marketing,"McCaskill 40, Akin 44",Akin +4,2012,akin,r,0.0
2014-02-13,NC,North Carolina Senate - Alexander vs. Hagan,PPP (D),"Alexander 45, Hagan 38",Alexander +7,2014,alexander,NaN,NaN
2014-03-11,NC,North Carolina Senate - Alexander vs. Hagan,PPP (D),"Alexander 45, Hagan 43",Alexander +2,2014,alexander,NaN,NaN
2014-04-02,NC,North Carolina Senate - Alexander vs. Hagan,SurveyUSA,"Alexander 46, Hagan 44",Alexander +2,2014,alexander,NaN,NaN


In [332]:
# Still have some errors in the RCP data
# Seems limited to doublecounting (at least at first glance)
sen_df_c[sen_df_c['Result'].str.contains('Huffman')]

Race               Poll  \
Date       State                                                         
2010-05-27 OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
           OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
2010-06-11 OR     Oregon Senate - Huffman vs. Wyden          SurveyUSA   
           OR     Oregon Senate - Huffman vs. Wyden          SurveyUSA   
2010-06-23 OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
           OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
2010-07-29 OR     Oregon Senate - Huffman vs. Wyden          SurveyUSA   
           OR     Oregon Senate - Huffman vs. Wyden          SurveyUSA   
           OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
           OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
2010-08-25 OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
           OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
2010-09-11 OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
           OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
2010-09-16 OR     Oregon Senate - Huffman vs. Wyden          SurveyUSA   
           OR     Oregon Senate - Huffman vs. Wyden          SurveyUSA   
2010-10-12 OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
           OR     Oregon Senate - Huffman vs. Wyden  Rasmussen Reports   
2010-10-16 OR     Oregon Senate - Huffman vs. Wyden          SurveyUSA   
           OR     Oregon Senate - Huffman vs. Wyden          SurveyUSA   

                                Result     Spread  Year   Lead Party  \
Date       State                                                       
2010-05-27 OR     Wyden 51, Huffman 38  Wyden +13  2010  wyden     d   
           OR     Wyden 51, Huffman 38  Wyden +13  2010  wyden     d   
2010-06-11 OR     Wyden 51, Huffman 38  Wyden +13  2010  wyden     d   
           OR     Wyden 51, Huffman 38  Wyden +13  2010  wyden     d   
2010-06-23 OR     Wyden 47, Huffman 37  Wyden +10  2010  wyden     d   
           OR     Wyden 47, Huffman 37  Wyden +10  2010  wyden     d   
2010-07-29 OR     Wyden 53, Huffman 35  Wyden +18  2010  wyden     d   
           OR     Wyden 53, Huffman 35  Wyden +18  2010  wyden     d   
           OR     Wyden 51, Huffman 35  Wyden +16  2010  wyden     d   
           OR     Wyden 51, Huffman 35  Wyden +16  2010  wyden     d   
2010-08-25 OR     Wyden 56, Huffman 36  Wyden +20  2010  wyden     d   
           OR     Wyden 56, Huffman 36  Wyden +20  2010  wyden     d   
2010-09-11 OR     Wyden 53, Huffman 35  Wyden +18  2010  wyden     d   
           OR     Wyden 53, Huffman 35  Wyden +18  2010  wyden     d   
2010-09-16 OR     Wyden 54, Huffman 38  Wyden +16  2010  wyden     d   
           OR     Wyden 54, Huffman 38  Wyden +16  2010  wyden     d   
2010-10-12 OR     Wyden 52, Huffman 36  Wyden +16  2010  wyden     d   
           OR     Wyden 52, Huffman 36  Wyden +16  2010  wyden     d   
2010-10-16 OR     Wyden 56, Huffman 34  Wyden +22  2010  wyden     d   
           OR     Wyden 56, Huffman 34  Wyden +22  2010  wyden     d   

                  Party Class  
Date       State               
2010-05-27 OR             1.0  
           OR             1.0  
2010-06-11 OR             1.0  
           OR             1.0  
2010-06-23 OR             1.0  
           OR             1.0  
2010-07-29 OR             1.0  
           OR             1.0  
           OR             1.0  
           OR             1.0  
2010-08-25 OR             1.0  
           OR             1.0  
2010-09-11 OR             1.0  
           OR             1.0  
2010-09-16 OR             1.0  
           OR             1.0  
2010-10-12 OR             1.0  
           OR             1.0  
2010-10-16 OR             1.0  
           OR             1.0

### Classified Congressional polling data

In [354]:
house_df_c = Party_Classification(house_df, all_win, states_dict)
house_df_c

Race  \
Date       State                                                      
2010-09-27 NJ            New Jersey 3rd District - Runyan vs. Adler   
2010-09-29 NJ            New Jersey 3rd District - Runyan vs. Adler   
2010-09-30 NJ            New Jersey 3rd District - Runyan vs. Adler   
2014-11-01 GA              Georgia 12th District - Allen vs. Barrow   
2014-11-03 GA              Georgia 12th District - Allen vs. Barrow   
2011-09-12 NV     Nevada 2nd District Special Election - Amodei ...   
           NV     Nevada 2nd District Special Election - Amodei ...   
           NV     Nevada 2nd District Special Election - Amodei ...   
           NV     Nevada 2nd District Special Election - Amodei ...   
2014-09-10 IA                   Iowa 3rd District - Young vs. Appel   
2010-09-21 NY             New York 24th District - Hanna vs. Arcuri   
2018-10-01 IA                    Iowa 3rd District - Young vs. Axne   
2018-10-28 IA                    Iowa 3rd District - Young vs. Axne   
2010-07-13 MN     Minnesota 6th District - Bachmann vs. Clark vs...   
           MN     Minnesota 6th District - Bachmann vs. Clark vs...   
2010-09-17 MN           Minnesota 6th District - Bachmann vs. Clark   
           MN           Minnesota 6th District - Bachmann vs. Clark   
2012-10-16 MN          Minnesota 6th District - Bachmann vs. Graves   
           MN          Minnesota 6th District - Bachmann vs. Graves   
2012-10-22 MN          Minnesota 6th District - Bachmann vs. Graves   
           MN          Minnesota 6th District - Bachmann vs. Graves   
2018-09-27 NE             Nebraska 2nd District - Bacon vs. Eastman   
2018-06-11 OH     Ohio 12th District Special Election - Balderso...   
2018-06-17 OH     Ohio 12th District Special Election - Balderso...   
2018-08-01 OH     Ohio 12th District Special Election - Balderso...   
2012-06-11 AZ     Arizona 8th District Special Election - Kelly ...   
           AZ     Arizona 8th District Special Election - Kelly ...   
2014-10-28 AZ             Arizona 2nd District - McSally vs. Barber   
           AZ             Arizona 2nd District - McSally vs. Barber   
2010-07-27 NM         New Mexico 1st District - Barela vs. Heinrich   
...                                                             ...   
2014-10-28 AK     Alaska At-Large District - Young vs. Dunbar vs...   
           AK     Alaska At-Large District - Young vs. Dunbar vs...   
           AK     Alaska At-Large District - Young vs. Dunbar vs...   
           AK     Alaska At-Large District - Young vs. Dunbar vs...   
2014-11-03 AK     Alaska At-Large District - Young vs. Dunbar vs...   
           AK     Alaska At-Large District - Young vs. Dunbar vs...   
           AK     Alaska At-Large District - Young vs. Dunbar vs...   
           AK     Alaska At-Large District - Young vs. Dunbar vs...   
2014-10-14 IA                   Iowa 3rd District - Young vs. Appel   
           IA                   Iowa 3rd District - Young vs. Appel   
2014-10-27 IA                   Iowa 3rd District - Young vs. Appel   
           IA                   Iowa 3rd District - Young vs. Appel   
2016-09-27 IA                  Iowa 3rd District - Young vs. Mowrer   
           IA                  Iowa 3rd District - Young vs. Mowrer   
2016-11-05 IA                  Iowa 3rd District - Young vs. Mowrer   
           IA                  Iowa 3rd District - Young vs. Mowrer   
2018-09-10 IA                    Iowa 3rd District - Young vs. Axne   
           IA                    Iowa 3rd District - Young vs. Axne   
           IA                    Iowa 3rd District - Young vs. Axne   
           IA                    Iowa 3rd District - Young vs. Axne   
2014-11-01 NY             New York 1st District - Zeldin vs. Bishop   
           NY             New York 1st District - Zeldin vs. Bishop   
2016-10-08 NY       New York 1st District - Zeldin vs. Throne-Holst   
           NY       New York 1st District - Zeldin vs. Throne-Holst   
2018-10-09 NY   

In [334]:
# Another example of errors in the RCP data
house_df_c[house_df_c['Result'].str.contains('Bachmann')]

Race  \
Date       State                                                      
2010-07-13 MN     Minnesota 6th District - Bachmann vs. Clark vs...   
           MN     Minnesota 6th District - Bachmann vs. Clark vs...   
2010-09-17 MN           Minnesota 6th District - Bachmann vs. Clark   
           MN           Minnesota 6th District - Bachmann vs. Clark   
2012-10-16 MN          Minnesota 6th District - Bachmann vs. Graves   
           MN          Minnesota 6th District - Bachmann vs. Graves   
2012-10-22 MN          Minnesota 6th District - Bachmann vs. Graves   
           MN          Minnesota 6th District - Bachmann vs. Graves   

                            Poll                             Result  \
Date       State                                                      
2010-07-13 MN          SurveyUSA  Bachmann 48, Clark 39, Anderson 6   
           MN          SurveyUSA  Bachmann 48, Clark 39, Anderson 6   
2010-09-17 MN          SurveyUSA              Bachmann 49, Clark 40   
           MN          SurveyUSA              Bachmann 49, Clark 40   
2012-10-16 MN     KSTP/SurveyUSA             Bachmann 50, Graves 41   
           MN     KSTP/SurveyUSA             Bachmann 50, Graves 41   
2012-10-22 MN       Star Tribune             Bachmann 51, Graves 45   
           MN       Star Tribune             Bachmann 51, Graves 45   

                       Spread  Year      Lead Party  Party Class  
Date       State                                                  
2010-07-13 MN     Bachmann +9  2010  bachmann     r          0.0  
           MN     Bachmann +9  2010  bachmann     r          0.0  
2010-09-17 MN     Bachmann +9  2010  bachmann     r          0.0  
           MN     Bachmann +9  2010  bachmann     r          0.0  
2012-10-16 MN     Bachmann +9  2012  bachmann     r          0.0  
           MN     Bachmann +9  2012  bachmann     r          0.0  
2012-10-22 MN     Bachmann +6  2012  bachmann     r          0.0  
           MN     Bachmann +6  2012  bachmann     r          0.0

## Edge cases (and not so edge cases)
* This is the best way to aggregate the data, but there are obvious issues
* We have no way to classify the party of the individual leading a poll unless that individual has held office before (shows up in the winners data)
* We get nan for party if the poll results are tied -- this may be okay as is

In [173]:
# Allen doesn't have a party because he's never won an election
sen_df_c.reset_index().set_index('Lead').loc['allen',].head()

,Date,State,Race,Poll,Result,Spread,Year,party,Party Class
Lead,,,,,,,,,
allen,2011-04-08,VA,Virginia Senate - Allen vs. Kaine,Roanoke College,"Allen 45, Kaine 32",Allen +13,2011,NaN,NaN
allen,2011-09-15,VA,Virginia Senate - Allen vs. Kaine,Quinnipiac,"Allen 45, Kaine 44",Allen +1,2011,NaN,NaN
allen,2011-09-25,VA,Virginia Senate - Allen vs. Kaine,Roanoke College,"Allen 42, Kaine 39",Allen +3,2011,NaN,NaN
allen,2011-12-21,VA,Virginia Senate - Allen vs. Kaine,Quinnipiac,"Kaine 42, Allen 44",Allen +2,2011,NaN,NaN
allen,2012-02-20,VA,Virginia Senate - Allen vs. Kaine,CNU/Times-Dispatch,"Kaine 40, Allen 42",Allen +2,2012,NaN,NaN


In [175]:
# Here's our output when there's a tie
sen_df_c[sen_df_c['Result'].str.contains('Rubio')].tail()

,,Race,Poll,Result,Spread,Year,Lead,party,Party Class
Date,State,,,,,,,,
2010-06-09,FL,Florida Senate - Rubio vs. Meek vs. Crist,Rasmussen Reports,"Crist 37, Rubio 37, Meek 15",Tie,2010,tie,NaN,NaN
2010-08-04,FL,Florida Senate - Rubio vs. Greene vs. Crist,AIF/McLaughlin (R),"Crist 37, Rubio 37, Greene 16",Tie,2010,tie,NaN,NaN
2016-06-29,FL,Florida Senate - Rubio vs. Murphy,Bay News 9/SurveyUSA,"Rubio 43, Murphy 43",Tie,2016,tie,NaN,NaN
2016-10-21,FL,Florida Senate - Rubio vs. Murphy,FOX 13/Opinion Savvy,"Rubio 46, Murphy 46",Tie,2016,tie,NaN,NaN
2016-10-30,FL,Florida Senate - Rubio vs. Murphy,Gravis,"Rubio 46, Murphy 46",Tie,2016,tie,NaN,NaN
